## Loading evaluation functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile as T
from skimage import io 

from scipy.spatial.distance import directed_hausdorff
from skimage.morphology import ball, binary_dilation
from skimage.metrics import hausdorff_distance
from skimage import io
from skimage.metrics import hausdorff_distance

In [ ]:
def batch_intersection_union(predict, target, nclass=2, thr=0.5):
    predict = predict > thr
    mini = 1
    maxi = nclass
    nbins = nclass
    predict = predict + 1
    target = target + 1

    predict = predict * (target > 0).astype(predict.dtype)
    intersection = predict * (predict == target)
    # areas of intersection and union
    area_inter, _ = np.histogram(intersection, bins=nbins, range=(mini, maxi))
    area_pred, _ = np.histogram(predict, bins=nbins, range=(mini, maxi))
    area_lab, _ = np.histogram(target, bins=nbins, range=(mini, maxi))
    area_union = area_pred + area_lab - area_inter
    assert (area_inter <= area_union).all(), \
        "Intersection area should be smaller than Union area"
    IoU = (np.float64(1.0) * area_inter / (np.spacing(1, dtype=np.float64) + area_union)).mean()
    return IoU

def batch_jaccard_index_and_dice_coefficient(predict, target, thr=0.5):
    predict = predict > thr
    predict = predict + 1
    target = target + 1

    tp = np.sum(((predict == 2) * (target == 2)) * (target > 0))
    fp = np.sum(((predict == 2) * (target == 1)) * (target > 0))
    fn = np.sum(((predict == 1) * (target == 2)) * (target > 0))
    tn = np.sum(((predict == 1) * (target == 1)) * (target > 0))

    ji = float(np.nan_to_num(tp / (tp + fp + fn)))
    dice = float(np.nan_to_num(2 * tp / (2 * tp + fp + fn)))
    return ji, dice

def batch_precision_recall(predict, target, thr=0.5):
    predict = predict > thr
    predict = predict + 1
    target = target + 1

    tp = np.sum(((predict == 2) * (target == 2)) * (target > 0))
    fp = np.sum(((predict == 2) * (target == 1)) * (target > 0))
    fn = np.sum(((predict == 1) * (target == 2)) * (target > 0))

    precision = float(np.nan_to_num(tp / (tp + fp)))
    recall = float(np.nan_to_num(tp / (tp + fn)))
    return precision, recall

def batch_sens_spec(predict, target, thr=0.5):
    predict = predict > thr
    predict = predict + 1
    target = target + 1

    tp = np.sum(((predict == 2) * (target == 2)) * (target > 0))
    fp = np.sum(((predict == 2) * (target == 1)) * (target > 0))
    tn = np.sum(((predict == 1) * (target == 1)) * (target > 0))
    fn = np.sum(((predict == 1) * (target == 2)) * (target > 0))

    sensitivity = float(np.nan_to_num(tp / (tp + fn)))
    specificity = float(np.nan_to_num(tn / (tn + fp)))
    return sensitivity, specificity

def batch_pix_accuracy(predict, target, thr=0.5):
    predict = predict > thr
    predict = predict + 1
    target = target + 1
    pixel_labeled = np.sum(target > 0)
    pixel_correct = np.sum((predict == target) * (target > 0))
    assert pixel_correct <= pixel_labeled, \
        "Correct area should be smaller than Labeled"
    acc = np.float64(1.0) * pixel_correct / (np.spacing(1, dtype=np.float64) + pixel_labeled)
    return acc

## Loading files

In [ ]:
PATH = r'C:\Users\sonil\PycharmProjects\Semester project\Data_output'

### Models

#### Segmentation

In [ ]:
#Load output data from 3DUnet in this example
#Tailor location of files for each model

#original label
img_path = '/home/slaguna/Documents/semproject/input_data/deepvess_slice/label_test/HaftJavaherian_DeepVess2018_GroundTruthLabel_25.tif'
img=np.divide(T.imread(img_path),np.amax(T.imread(img_path)))

#output prediction
out_path = '/home/slaguna/Documents/semproject/outputs/3dunet_DV_output.tif'
out=T.imread(out_path)

f, axarr = plt.subplots(1,2,figsize=(15,15))
axarr[0].imshow(img[90,:])
axarr[0].set_title('label original')
axarr[1].imshow(out[90,:])
axarr[1].set_title('predicted image')

#### Loss

In [ ]:
#Load stored loss
loss=np.load(r'C:\Users\sonil\PycharmProjects\Semester_project\Data_output\loss\loss_DV_inhouse.npy')
plt.plot(loss)
plt.title('Loss: DeepVess architecture trained on InHouse data')
plt.show()

## Evaluating segmentations

In [ ]:
 batch_intersection_union(out, img)

In [ ]:
batch_jaccard_index_and_dice_coefficient(out, img)

In [ ]:
batch_precision_recall(out,img)

In [ ]:
f1_score(out, img)

In [ ]:
batch_sens_spec(out, img)

In [ ]:
batch_pix_accuracy(out, img)

## Evaluating skeleton

In [ ]:
#Location of extracted skeleton, i.e. coming from TopoDV model
skel = io.imread(r'C:\Users\sonil\PycharmProjects\Semester_project\Data_output\skeleton\skeleton_IH_topoDV_25.tif')
#skel = io.imread(r'C:\Users\sonil\PycharmProjects\Semester project\Data_output\skeleton\skeleton_IH_UMIS_25.tif')
#skel = io.imread(r'C:\Users\sonil\PycharmProjects\Semester project\Data_output\skeleton\skeleton_25_trainedDV.tif')

#Groundtruth skeleton location 
targ = io.imread(r'C:\Users\sonil\PycharmProjects\Semester_project\Data_output\skeleton\skeleton_test25.tif')


In [ ]:
#First dilation to avoid outliers Dice score
skel=binary_dilation(skel, ball(5))
targ=binary_dilation(targ, ball(5))

print('Jaccard index and Dice Score:' , batch_jaccard_index_and_dice_coefficient(skel, targ) )
print('Hausdorff distance:', hausdorff_distance(skel,targ))
